In [1]:
import deepchem as dc
import numpy as np

/home/farhanhubble/anaconda3/envs/deepchem/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Load the Tox21 Dataset.
The [Tox21](https://ncats.nih.gov/tox21/about/goals) (Toxicity in 21st century) dataset maps molecules, drugs and environmental chemicals, to their toxicity towards 12 enzymes. 

The `molnet` module in **deepchem** library has a function `load_tox21()` that loads the dataset. The function returns three items -- a task list, a list of datasets and a list of transformers. 

In [2]:
tox21_tasks, tox21_datasets, transformers = dc.molnet.load_tox21()

Loading raw samples now.
shard_size: 8192
About to start loading CSV from /tmp/tox21.csv.gz
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
Featurizing sample 6000
Featurizing sample 7000
TIMING: featurizing shard 0 took 11.673 s
TIMING: dataset construction took 12.363 s
Loading dataset from disk.
TIMING: dataset construction took 0.798 s
Loading dataset from disk.
TIMING: dataset construction took 0.724 s
Loading dataset from disk.
TIMING: dataset construction took 0.372 s
Loading dataset from disk.
TIMING: dataset construction took 0.365 s
Loading dataset from disk.


The task list is a list of experiments from which the data in the datasets has been collected. The names of the experiments would probably make sense to a bilogist, but are just cryptic acronyms for a computer scientist. In my understanding, each of the 12 tasks represents an experiment where all molecules were tested for their toxicity towards a specific enzyme.

In [3]:
tox21_tasks

['NR-AR',
 'NR-AR-LBD',
 'NR-AhR',
 'NR-Aromatase',
 'NR-ER',
 'NR-ER-LBD',
 'NR-PPAR-gamma',
 'SR-ARE',
 'SR-ATAD5',
 'SR-HSE',
 'SR-MMP',
 'SR-p53']

The dataset object is a list of 3 items, each of the type `deepchem.data.datasets.DiskDatase`, representing the train, validation and test sets that have been saved to disk for reuse.

In [4]:
tox21_datasets

(<deepchem.data.datasets.DiskDataset at 0x7fb4a6db4a58>,
 <deepchem.data.datasets.DiskDataset at 0x7fb4a6e54e48>)

In [5]:
train_dataset, valid_dataset, test_dataset = tox21_datasets

Inside every dataset, there is a varible X. A row in X represents a molecule using 1024 features, extracted from a molecule's graph structure, using [graph convolution](https://tkipf.github.io/graph-convolutional-networks/).

In [6]:
train_dataset.X.shape

(6264, 1024)

In [7]:
valid_dataset.X.shape

(783, 1024)

In [8]:
test_dataset.X.shape

(784, 1024)

Every dataset also has a variable y that records the chemical's toxicity towards each of the 12 Enzymes as either 1 (toxic) or 0 non-toxic.

In [9]:
train_dataset.y.shape

(6264, 12)

In [10]:
valid_dataset.y.shape

(783, 12)

In [11]:
test_dataset.y.shape

(784, 12)

There is a caveat,though. Some molecules have not been tested for their toxicity towards some of the enzymes. The `y` value for such a molecule-enzyme pair is meaningless. An additional matrix `w` supresses such pairs from any computation.

In [16]:
train_dataset.w.shape

(6264, 12)

In [17]:
np.count_nonzero(train_dataset.w)

62166

The matrix `w` is dot multiplied with the matrix `y`. 

In [25]:
train_dataset.w[:10]

array([[ 1.        ,  1.        ,  7.52734375,  0.        ,  0.        ,
         1.        ,  1.        ,  5.1910828 ,  1.        ,  1.        ,
         1.        ,  1.        ],
       [ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         1.        ,  1.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  1.        ,
         0.        ,  0.        ],
       [ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         1.        ,  1.        ],
       [ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ],
       [ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  

`w` has another function too. It (roughly) balances the weight of each class across the dataset, which can be verified by printing the sum of each column in `w`.

In [34]:
train_dataset.w.sum(axis=0)

array([10989.22977346, 10472.83544304,  9220.734375  ,  8776.99333333,
        8587.03783102, 10308.67142857, 10003.93548387,  7876.17834395,
       10889.03030303,  9718.40053763,  7825.78431373, 10365.0212766 ])

The third and last item returned by `load_tox21()` is a list of transformers -- functions that have been used to transform from raw data to the data in the datasets. In this case there is only one element in the list, the transformer for balancing the (labels in the) dataset. Indeed the matrix `w` has been populated by this transformer function.

In [18]:
transformers

The **deepchem** library comes with built-in classifiers like the MultiTaskClassifier that performs [multi-label classification](https://en.wikipedia.org/wiki/Multi-label_classification). To build a simple classifier, using this classifier as the base, we need to specify the number of labels (n_tasks), the shape of the input feature vector for a molecule (n_features) and the a list contianing the sizes of the hidden layers. We build a simple netwrk with just one hidden layer with 1000 neurons.

In [38]:
model = dc.models.MultiTaskClassifier(n_tasks=12, n_features=1024, layer_sizes=[1000] )

In [39]:
model.fit(train_dataset)

859.3929135277158

Finally we use the `Metric` class to create a metric that measures the ROC-AUC score. Since ROC-AUC is for binary classification, when creating the metric object, we specify an averaging strategy using `np.mean()` here.

In [21]:
metric = dc.metrics.Metric(metric=dc.metrics.roc_auc_score, task_averager=np.mean)

In [24]:
model.evaluate(train_dataset,[metric], transformers)

computed_metrics: [0.989502015028443, 0.9963343469010919, 0.9602251318096955, 0.9807183278132723, 0.9021875727251571, 0.9833791726186669, 0.9918846595972077, 0.9097783970609157, 0.9877642970984737, 0.9702538035841299, 0.9469223127885017, 0.9761486826648766]


{'mean-roc_auc_score': 0.9662582266408694}

In [23]:
model.evaluate(test_dataset,[metric], transformers)

computed_metrics: [0.7949455819959417, 0.8457258472432979, 0.9025269218466787, 0.800345405065941, 0.7152661278023486, 0.7969969969969971, 0.7174776564051638, 0.7158706725567978, 0.8630527817403709, 0.7123846407382992, 0.8704273362154973, 0.7840447154471546]


{'mean-roc_auc_score': 0.793255390337874}